# Texas school accountability data
This notebook has the scripts needed to cut, filter and analyze school accountability data from the Texas Education Association.

### Download the data
Accountability data for 2013-2016 are in the `data` folder inside this repo. Here's how you could get them yourself, using the 2015 data file as an example.

First: Go to the [accountability data portal](https://rptsvr1.tea.texas.gov/perfreport/account/2015/) and click the "Download data" link on the left rail.

<img src="img/1-portal-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

On the resulting page, click the "Campus-level Data" radio button, then scroll down and click "Continue."

<img src="img/2-data-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

Finally, on the data download page, select "Tab delimited" from the select menu. Click the "Select all" button. Then click the "Download" button.

<img src="img/3-download-page.gif" style="border: 1px solid #ccc; margin: 20px auto 40px auto;" />

I renamed this file `2015-tx-school-acc-data.dat` and dropped it into the `/data` folder, then repeated this process for 2014 and 2013.

Also, I snagged the file layouts ([e.g.](https://rptsvr1.tea.texas.gov/perfreport/account/2015/download/camprate.html)) and saved them as .tsv files in the `/data` directory. In practice, however, they didn't always match up with the data, so I used them as a rough guide and consulted a sample of published summary reports [like this one](https://rptsvr1.tea.texas.gov/perfreport/account/2013/static/summary/campus/c227901170.pdf) to check expected values against actual values.

Also also, I grabbed a .csv file with [spatial and contact data for every school in Texas](http://schoolsdata.tea-texas.opendata.arcgis.com/datasets/059432fd0dcb4a208974c235e837c94f_0), renamed the columns I'm going to use later (`campus_id`, `city`, `lat`, `lng`, `district_id`) and saved it as `/data/school_location_data.csv`. (TODO: grab the [districts shapefile](http://schoolsdata.tea-texas.opendata.arcgis.com/datasets/e115fed14c0f4ca5b942dc3323626b1c_0), too.)

### Process preliminary 2016 data

In [2]:
import csv

with open('data/2016-raw-data.csv', 'r') as file_in, \
         open('data/2016-processed-data.txt', 'w') as file_out:
    reader = csv.reader(file_in, delimiter=',')
    
    """
    fieldnames_for_later = ['campus_id', 'campus_name', 'campus_population', 'campus_pct_disadvantaged', 
                            'campus_pct_english_language_learners', 'district_name', 'index1_target_score',
                            'index1_score', 'index2_target_score', 'index2_score', 'index3_target_score',
                            'index3_score', 'index4_target_score', 'index4_score', 'distinction_reading',
                            'distinction_math', 'distinction_student_progress', 'distinction_science',
                            'distinction_social_studies', 'distinction_close_performance_gap',
                            'distinction_postsecondary_readiness', 'jjaep', 'daep', 'year', 'overall_rating',
                            'updated_rating']
    """

    fieldnames = ['campus_id', 'campus_name', 'district_name', 'rating', 'i1_target', 'i1_score',
                  'i2_target', 'i2_score', 'i3_target', 'i3_score', 'i4_target', 'i4_score', 'year']
    
    writer = csv.DictWriter(file_out, fieldnames=fieldnames, delimiter="|")
    # writer.writeheader()
    
    for row in reader:
        if row[1] != "":
            d = {}
            d['campus_id'] = row[3].zfill(9)
            d['campus_name'] = row[1]
            d['district_name'] = row[0]
            d['rating'] = row[5]
            d['i1_target'] = row[7]
            d['i1_score'] = row[6]
            d['i2_target'] = row[10]
            d['i2_score'] = row[9]
            d['i3_target'] = row[13]
            d['i3_score'] = row[12]
            d['i4_target'] = row[16]
            d['i4_score'] = row[15]
            d['year'] = '2016'
            writer.writerow(d)

print("done")

done


### Cut and stack
So now I can use `awk` and `csvkit` to extract the columns I need from each file and append them to `data/stacked-file.csv`. (The file layouts are different each year.) Then I joined a few columns of location data and sorted by campus ID.

In [3]:
%%bash
# truncate existing file
# :> data/stacked_data.csv

# write headers for full data
# echo "campus_id,campus_name,campus_population,campus_pct_disadvantaged,campus_pct_english_language_learners,district_name,index1_target_score,index1_score,index2_target_score,index2_score,index3_target_score,index3_score,index4_target_score,index4_score,distinction_reading,distinction_math,distinction_student_progress,distinction_science,distinction_social_studies,distinction_close_performance_gap,distinction_postsecondary_readiness,jjaep,daep,year,overall_rating,updated_rating" >> data/stacked_data.csv

# 2013 data
# awk -F '\t' '{OFS=","; if (NR!=1) {print $1,$6,$44,$46,$48,$51,$20,$19,$25,$24,$30,$29,$35,$34,$5,$3,$4,".",".",".",".",$12,$11,"2013",$49,$50;}}' data/2013-tx-school-acc-data.dat >> data/stacked_data.csv

# 2014 data
# awk -F '\t' '{OFS=","; if (NR!=1) {print $1,$9,$49,$51,$53,$56,$23,$22,$28,$27,$33,$32,$39,$37,$6,$3,$5,$7,$8,$2,$4,$15,$13,"2014",$54,$55;}}' data/2014-tx-school-acc-data.dat >> data/stacked_data.csv

# 2015 data
# awk -F '\t' '{OFS=","; if (NR!=1) {print $1,$9,$49,$51,$53,$56,$23,$22,$28,$27,$33,$32,$38,$37,$6,$3,$5,$7,$8,$2,$4,$15,$13,"2015",$54,$55;}}' data/2015-tx-school-acc-data.dat >> data/stacked_data.csv

# 2016 data
# TODO: come back when detail data is posted and fill in

# join to location data and sort by campus ID
# csvcut -c 9,7,2,1,15 data/school_location_data.csv | csvjoin -c "campus_id,campus_id" data/stacked_data.csv - | csvcut -c 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31 | csvsort -c 1 > data/stacked_data_with_coordinates.csv

# check for ish
# csvclean -n data/stacked_data_with_coordinates.csv

# report line count
# wc -l data/stacked_data_with_coordinates.csv


####  get slim version of data for 2016 interactive ####

# truncate existing
:> data/stacked_data_slim.csv

# write headers
echo "campus_id|campus_name|district_name|rating|i1_target|i1_score|i2_target|i2_score|i3_target|i3_score|i4_target|i4_score|year" >> data/stacked_data_slim.csv

# slim version of 2013 data
awk -F '\t' '{OFS="|"; if (NR!=1) {print$1,$6,$51,$49,$20,$19,$25,$24,$30,$29,$35,$34,"2013"}}' data/2013-tx-school-acc-data.dat >> data/stacked_data_slim.csv

# slim version of 2014 data
awk -F '\t' '{OFS="|"; if (NR!=1) {print $1,$9,$56,$54,$23,$22,$28,$27,$33,$32,$39,$37,"2014"}}' data/2014-tx-school-acc-data.dat >> data/stacked_data_slim.csv

# slim version of 2015 data
awk -F '\t' '{OFS="|"; if (NR!=1) {print $1,$9,$56,$54,$23,$22,$28,$27,$33,$32,$38,$37,"2015"}}' data/2015-tx-school-acc-data.dat >> data/stacked_data_slim.csv

# 2016 data
cat data/2016-processed-data.txt >> data/stacked_data_slim.csv

csvclean -n data/stacked_data_slim.csv

No errors.


_Psst, future me_: I created a Python dict with a 1-indexed column layout for each year of data at `/col_index.py`.
You're welcome.

### Load up the data to analyze
Time to analyze some data. Should I use `R`, or `numpy`, or maybe `pandas`?

<img src="img/achewood.png" style="margin: 0;" />

Ha ha OK guys, settle down, I'll use `Agate`. First, create a table.

In [4]:
import agate

"""
# Define the column types
column_types = {
    'campus_id': agate.Text(),
    'campus_name': agate.Text(),
    'campus_population': agate.Number(),
    'campus_pct_disadvantaged': agate.Number(),
    'campus_pct_english_language_learners': agate.Number(),
    'district_name': agate.Text(),
    'index1_target_score': agate.Number(),
    'index1_score': agate.Number(),
    'index2_target_score': agate.Number(),
    'index2_score': agate.Number(),
    'index3_target_score': agate.Number(),
    'index3_score': agate.Number(),
    'index4_target_score': agate.Number(),
    'index4_score': agate.Number(),
    'distinction_reading': agate.Boolean(),
    'distinction_math': agate.Boolean(),
    'distinction_student_progress': agate.Boolean(),
    'distinction_science': agate.Boolean(),
    'distinction_social_studies': agate.Boolean(),
    'distinction_close_performance_gap': agate.Boolean(),
    'distinction_postsecondary_readiness': agate.Boolean(),
    'jjaep': agate.Boolean(),
    'daep': agate.Boolean(),
    'year': agate.Text(),
    'overall_rating': agate.Text(),
    'updated_rating': agate.Number(),
    'district_id': agate.Text(),
    'lng': agate.Number(),
    'lat': agate.Number(),
    'city': agate.Text()
}

school_ratings = agate.Table.from_csv('data/stacked_data_with_coordinates.csv', column_types=column_types)

print(school_ratings)
"""

### slim version ###
column_types = {    
    'campus_id': agate.Text(),
    'campus_name': agate.Text(),
    'rating': agate.Text(),
    'district_name': agate.Text(),
    'i1_target': agate.Number(),
    'i1_score': agate.Number(),
    'i2_target': agate.Number(),
    'i2_score': agate.Number(),
    'i3_target': agate.Number(),
    'i3_score': agate.Number(),
    'i4_target': agate.Number(),
    'i4_score': agate.Number(),
    'year': agate.Text(),
}

school_ratings = agate.Table.from_csv('data/stacked_data_slim.csv', column_types=column_types, delimiter="|")

print(school_ratings)

|----------------+------------|
|  column        | data_type  |
|----------------+------------|
|  campus_id     | Text       |
|  campus_name   | Text       |
|  district_name | Text       |
|  rating        | Text       |
|  i1_target     | Number     |
|  i1_score      | Number     |
|  i2_target     | Number     |
|  i2_score      | Number     |
|  i3_target     | Number     |
|  i3_score      | Number     |
|  i4_target     | Number     |
|  i4_score      | Number     |
|  year          | Text       |
|----------------+------------|



### Process the data
I need to:
* Exclude disciplinary alternative schools ("daep") and kid jails ("jjaep").
* Run the campus names through some text transforms to standardize names.

In [5]:
import re

TEXT_TRANSFORMS = (
    (r"H S$", "High School"),
    (r"MIDDLE$", "Middle School"),
    (r"JR H S$", "Junior High School"),
    (r"INT$", "Intermediate"),
    (r"EL$", "Elementary")
)

def clean_text(garb):
    if garb:
        for item in TEXT_TRANSFORMS:
            garb = re.sub(*item, garb, flags=re.IGNORECASE)
        return garb.title()

# drop disciplinary schools
# school_ratings_no_disc = school_ratings.where(
#     lambda row: row['jjaep'] is False and row['daep'] is False
#)

# clean up text
school_ratings_cleaned = school_ratings.compute([
    ('campus_name', agate.Formula(agate.Text(), lambda row: clean_text(row['campus_name']))),
    # ('city', agate.Formula(agate.Text(), lambda row: clean_text(row['city']))),
    ('district_name', agate.Formula(agate.Text(), lambda row: clean_text(row['district_name'])))
], replace=True)

"""
disciplinary_schools_count = len(school_ratings.rows) - len(school_ratings_cleaned.rows)

print(
    "Chopped",
    "{:,}".format(disciplinary_schools_count),
    "disciplinary schools ..."
)
"""

'\ndisciplinary_schools_count = len(school_ratings.rows) - len(school_ratings_cleaned.rows)\n\nprint(\n    "Chopped",\n    "{:,}".format(disciplinary_schools_count),\n    "disciplinary schools ..."\n)\n'

### How did local schools do this year?

In [6]:
# csvcut -d "," -c 6 data/stacked_data_with_coordinates.csv | sort | uniq > districts.txt

local_districts = ["AUSTIN ACHIEVE PUBLIC SCHOOLS", "AUSTIN DISCOVERY SCHOOL", "AUSTIN ISD", "ROUND ROCK ISD", "LEANDER ISD", "PFLUGERVILLE ISD", "HAYS CISD", "GEORGETOWN ISD", "BASTROP ISD", "MANOR ISD", "LAKE TRAVIS ISD", "EANES ISD", "SAN MARCOS CISD", "HUTTO ISD", "DRIPPING SPRINGS ISD", "DEL VALLE ISD"]

local_school_data = school_ratings_cleaned.where(
    lambda row: row['district_name'].upper() in local_districts
)

year_to_check = '2016'

grouped_by_district = local_school_data.group_by('campus_id')

for table in grouped_by_district:
    print(table.rows.keys())

"""

"DIST_NAME"
"X of X schools met standard in 2016; X reported no data"


print(
    "Pulled data for",
    "{:,}".format(len(local_school_data.rows)),
    "schools in",
    "{:,}".format(len(local_districts)),
    "local districts ..."
)
"""

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


'\n\n"DIST_NAME"\n"X of X schools met standard in 2016; X reported no data"\n\n\nprint(\n    "Pulled data for",\n    "{:,}".format(len(local_school_data.rows)),\n    "schools in",\n    "{:,}".format(len(local_districts)),\n    "local districts ..."\n)\n'

### Have any schools that reported scores for all 4 standards missed every one?

In [7]:
def sad_trombone(row):
    if row['index1_score'] and \
            row['index1_target_score'] and \
            row['index2_score'] and \
            row['index2_target_score'] and \
            row['index3_score'] and \
            row['index3_target_score'] and \
            row['index4_score'] and \
            row['index4_target_score']:
        return row['index1_score'] < row['index1_target_score'] and \
        row['index2_score'] < row['index2_target_score'] and \
        row['index3_score'] < row['index3_target_score'] and \
        row['index4_score'] < row['index4_target_score']

missed_erry_one = school_ratings_cleaned.where(
    lambda row: sad_trombone(row)
).order_by('campus_id')

print(
    len(missed_erry_one.rows),
    "schools missed every one:\n"
)

for row in missed_erry_one.rows:
    print(
        row['campus_name'],
        "\n" + row['district_name'],
        "\n" + row['city'],
        "\n" + row['year'],
        "\n" + "https://rptsvr1.tea.texas.gov/perfreport/account/{}/static/summary/campus/c{}.pdf".format(row['year'], row['campus_id']),
        "\n"
    )

KeyError: 'index1_score'

### Dump to an out dict

In [80]:
from operator import itemgetter
import json

outdict = {}
index_list = []

RATINGS = (
    (r"Met Standard", "M"),
    (r"Met Standard\**", "M"),
    (r"Met Standard-Paired", "M"),
    (r"Not Rated", "X"),
    (r"Not Rated: Data Integrity Issues", "X"),
    (r"Not Rated: Data Integrity Issues-Paired", "X"),
    (r"^Z$", "X"),
    (r"^Q$", "X"),
    (r"^T$", "X"),
    (r"Improvement Required-Paired", "I"),
    (r"Improvement Required", "I"),
    (r"Met Alternative Standard-Paired", "A"),
    (r"Met Alternative Standard", "A")
)


def clean_ratings(garb):
    if garb:
        for item in RATINGS:
            garb = re.sub(*item, garb, flags=re.IGNORECASE)
        return garb

def de_decimalize(num, type_method):
    """Turn decimals into something JSON-serializable."""
    if num:
        try:
            return type_method(num)
        except:
            return num

for row in school_ratings_cleaned.rows:
    rating = {}
    index1 = {}
    index2 = {}
    index3 = {}
    index4 = {}
    
    rating['year'] = row['year']
    rating['rating'] = clean_ratings(row['rating'])

    index1['year'] = row['year']
    index1['target'] = de_decimalize(row['i1_target'], int)
    index1['score'] = de_decimalize(row['i1_score'], int)

    index2['year'] = row['year']
    index2['target'] = de_decimalize(row['i2_target'], int)
    index2['score'] = de_decimalize(row['i2_score'], int)

    index3['year'] = row['year']
    index3['target'] = de_decimalize(row['i3_target'], int)
    index3['score'] = de_decimalize(row['i3_score'], int)

    index4['year'] = row['year']
    index4['target'] = de_decimalize(row['i4_target'], int)
    index4['score'] = de_decimalize(row['i4_score'], int)

    d = outdict.get(row['campus_id'], None)

    if not d:
        outdict[row['campus_id']] = {}
        outdict[row['campus_id']]['name'] = row['campus_name']
        outdict[row['campus_id']]['dist_name'] = row['district_name']
        outdict[row['campus_id']]['ratings'] = []    
        
    idx1 = outdict[row['campus_id']].get('1', None)
    idx2 = outdict[row['campus_id']].get('2', None)
    idx3 = outdict[row['campus_id']].get('3', None)
    idx4 = outdict[row['campus_id']].get('4', None)

    if not idx1:
        outdict[row['campus_id']]['1'] = {}
        outdict[row['campus_id']]['1']['scores'] = []

    if not idx2:
        outdict[row['campus_id']]['2'] = {}
        outdict[row['campus_id']]['2']['scores'] = []

    if not idx3:
        outdict[row['campus_id']]['3'] = {}
        outdict[row['campus_id']]['3']['scores'] = []

    if not idx4:
        outdict[row['campus_id']]['4'] = {}
        outdict[row['campus_id']]['4']['scores'] = []
        
    outdict[row['campus_id']]['1']['scores'].append(index1)
    outdict[row['campus_id']]['2']['scores'].append(index2)
    outdict[row['campus_id']]['3']['scores'].append(index3)
    outdict[row['campus_id']]['4']['scores'].append(index4)
    outdict[row['campus_id']]['ratings'].append(rating)
    
# fill in missing years
expected_years = ['2013', '2014', '2015', '2016']

for school in outdict:
    years = [x['year'] for x in outdict[school]['ratings']]
    missing_years = [x for x in expected_years if x not in years]
    
    if len(missing_years) > 0:
        for missing_year in missing_years:
            outdict[school]['ratings'].append({"year": missing_year, "rating": None})

    for i in range(1,5):
        years = [x['year'] for x in outdict[school][str(i)]['scores']]
        missing_years = [x for x in expected_years if x not in years]
        if len(missing_years) > 0:
            for missing_year in missing_years:
                outdict[school][str(i)]['scores'].append({'target': None, 'year': missing_year, 'score': None})
                
    # while we're in here, sort the list of dicts by year
    outdict[school]['ratings'] = sorted(outdict[school]['ratings'], key=itemgetter('year'))
    outdict[school]['1']['scores'] = sorted(outdict[school]['1']['scores'], key=itemgetter('year'))
    outdict[school]['2']['scores'] = sorted(outdict[school]['2']['scores'], key=itemgetter('year'))
    outdict[school]['3']['scores'] = sorted(outdict[school]['3']['scores'], key=itemgetter('year'))
    outdict[school]['4']['scores'] = sorted(outdict[school]['4']['scores'], key=itemgetter('year'))
    
    # write the record to its own file and add to index
    with open("public/assets/data/" + school + ".json", "w") as f:
        f.write(json.dumps(outdict[school]))
    
    index_list.append({
        "name": outdict[school]['name'],
        "id": school,
        "district": outdict[school]['dist_name']
    })


with open('public/assets/data/search_index.json', 'w') as f:
    f.write(json.dumps(index_list))
    
print("Wrote", "{:,}".format(len(index_list)), "records to file.")

Wrote 9,135 records to file.


### Peep data on local schools

In [ ]:
# csvcut -d "," -c 6 data/stacked_data_with_coordinates.csv | sort | uniq > districts.txt

"""
local_districts = ["AUSTIN ACHIEVE PUBLIC SCHOOLS", "AUSTIN DISCOVERY SCHOOL", "AUSTIN ISD", "ROUND ROCK ISD", "LEANDER ISD", "PFLUGERVILLE ISD", "HAYS CISD", "GEORGETOWN ISD", "BASTROP ISD", "MANOR ISD", "LAKE TRAVIS ISD", "EANES ISD", "SAN MARCOS CISD", "HUTTO ISD", "DRIPPING SPRINGS ISD", "DEL VALLE ISD"]

local_school_data = school_ratings_transformed.where(
    lambda row: row['district_name'].upper() in local_districts
)

print(
    "Pulled data for",
    "{:,}".format(len(local_school_data.rows)),
    "schools in",
    "{:,}".format(len(local_districts)),
    "local districts ..."
)

"""